# Ridge and Lasso Regression

Data - https://drive.google.com/file/d/1ItJT3faKTNi4eZNo3GGnUORDO2_7C6JZ/view?usp=sharing

## 1. Ridge and Lasso Regression - Scratch - Using Gradient Descent 

In [1]:
import numpy as np

class RidgeAndLassoRegressionScratchGD:
    
    def __init__(self, norm = 'L2', alpha = 0.001,iterations = 2000, learning_rate=0.01, option='BatchGD'):
        self.coef_ = None
        self.intercept_ = None
        self.iterations = iterations
        self.learning_rate = learning_rate
        # norm could be L2 for Ridge and L1 for Lasso
        self.norm = norm,
        # alpha is the hyper parameter of Regularisation
        self.alpha = alpha
        # option could be 'BatchGD', 'MiniBatchGD', 'SGD'
        self.option = option
    
    def fit(self,X,y):
        X = np.array(X)
        y = np.array(y)
        weights = np.array([np.random.randn(1)[0] for i in range(len(X[0])+1)])
        ## making sure none of weights are 0, replacing them with 0.1
        weights = np.where(weights == 0, 0.1, weights)
        X = np.array([[1]+list(i) for i in X])
        if(self.option == 'MiniBatchGD'):
            indices = [np.random.randint(len(X)) for i in range(int(0.1*len(X)))]
            X_new = X[indices]
            y_new = y[indices]
        elif(self.option == 'SGD'):
            indices = [np.random.randint(len(X))]
            X_new = X[indices]
            y_new = y[indices]
        else:
            X_new = X
            y_new = y
        y_err = y_new - (weights @ X_new.T)
        if(self.norm == 'L1'):
            y_err -= self.alpha * sum(np.abs(weights))
        else:
            y_err -= self.alpha * sum(weights**2)
        for i in range(self.iterations):
            if(self.norm == 'L1'):
                modified_X = np.column_stack([X_new[:, i] + self.alpha if weights[i] > 0 else X_new[:, i] - alpha for i in range(1,len(X_new[0]))])
                modified_X = np.array([[1]+list(i) for i in modified_X])
            else:
                modified_X = np.column_stack([X_new[:, i] + (2*self.alpha*weights[i]) for i in range(1,len(X_new[0]))])
                modified_X = np.array([[1]+list(i) for i in modified_X])
            weights += (2*self.learning_rate*(y_err @ modified_X))/len(modified_X)
            y_err = y_new - (weights @ X_new.T)
            if(self.norm == 'L1'):
                y_err -= self.alpha * sum(np.abs(weights))
            else:
                y_err -= self.alpha * sum(weights**2)
            if(self.option == 'MiniBatchGD'):
                indices = [np.random.randint(len(X)) for i in range(int(0.1*len(X)))]
                X_new = X[indices]
                y_new = y[indices]
            elif(self.option == 'SGD'):
                indices = [np.random.randint(len(X))]
                X_new = X[indices]
                y_new = y[indices]
            else:
                X_new = X
                y_new = y
        self.intercept_ = np.array([weights[0]])
        self.coef_ = weights[1:]
        self.weights = weights
    
    def predict(self,X):
        X = np.array(X)
        newX = []
        for i in X:
            newX.append([1]+list(i))
        X = np.array(newX)
        return X @ self.weights.T

In [2]:
import pandas as pd
link = 'https://drive.google.com/file/d/1ItJT3faKTNi4eZNo3GGnUORDO2_7C6JZ/view?usp=sharing'
df = pd.read_csv(f'https://drive.google.com/uc?id={link.split("/")[-2]}')
df.head()

,cgpa,package
0,6.89,3.26
1,5.12,1.98
2,7.82,3.25
3,7.42,3.67
4,6.94,3.57


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('package',axis=1),df['package'],test_size=0.2,random_state=42)

In [4]:
lasso_scratch_gd = RidgeAndLassoRegressionScratchGD(norm = 'L1', alpha = 0.01, iterations=10000, learning_rate=0.01, option='BatchGD')
ridge_scratch_gd = RidgeAndLassoRegressionScratchGD(norm = 'L2', alpha = 0.01, iterations=10000, learning_rate=0.01, option='BatchGD')

In [5]:
lasso_scratch_gd.fit(X_train, y_train)
ridge_scratch_gd.fit(X_train, y_train)

In [6]:
print('Lasso coeff',lasso_scratch_gd.coef_)
print('Ridge coeff',ridge_scratch_gd.coef_)

Lasso coeff [0.57345137]
Ridge coeff [0.57333071]


In [7]:
print('Lasso intercept',lasso_scratch_gd.intercept_)
print('Ridge intercept',ridge_scratch_gd.intercept_)

Lasso intercept [-1.03523961]
Ridge intercept [-1.03435466]


In [8]:
lasso_scratch_gd_preds = lasso_scratch_gd.predict(X_test)
ridge_scratch_gd_preds = ridge_scratch_gd.predict(X_test)

In [9]:
from sklearn.metrics import r2_score
print("Lasso R2 Score",r2_score(lasso_scratch_gd_preds,y_test))
print("Ridge R2 Score",r2_score(ridge_scratch_gd_preds,y_test))

Lasso R2 Score 0.7287460969526806
Ridge R2 Score 0.7286673028544517


## 2. Ridge and Lasso Regression Sklearn Library - Gradient Descent

In [10]:
from sklearn.linear_model import Lasso, Ridge

In [11]:
ridge_model = Ridge(alpha=0.01)
ridge_model.fit(X_train, y_train)

lasso_model = Lasso(alpha=0.01)
lasso_model.fit(X_train, y_train)

Lasso(alpha=0.01)

In [12]:
print('Lasso coeff',lasso_model.coef_)
print('Ridge coeff',ridge_model.coef_)

Lasso coeff [0.56579315]
Ridge coeff [0.5742261]


In [13]:
print('Lasso intercept',lasso_model.intercept_)
print('Ridge intercept',ridge_model.intercept_)

Lasso intercept -0.967718167048357
Ridge intercept -1.0267941552084712


In [14]:
lasso_model_preds = lasso_model.predict(X_test)
ridge_model_preds = ridge_model.predict(X_test)

In [15]:
from sklearn.metrics import r2_score
print("Lasso R2 Score",r2_score(lasso_model_preds,y_test))
print("Ridge R2 Score",r2_score(ridge_model_preds,y_test))

Lasso R2 Score 0.7270482374488476
Ridge R2 Score 0.7337743970757031
